<a href="https://colab.research.google.com/github/therealhaseeb/Email-Classification/blob/main/email_classification_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import keras
from keras.layers import Input, Dense, Dropout, LSTM, Bidirectional
from keras import Model
import pandas as pd
import re
import random
import email
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics 
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from gensim.models.word2vec import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import files 
uploaded = files.upload() 
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [3]:
!kaggle datasets download -d wcukierski/enron-email-dataset
!unzip /content/enron-email-dataset.zip
!rm /content/enron-email-dataset.zip

 97% 346M/358M [00:02<00:00, 199MB/s]
100% 358M/358M [00:02<00:00, 167MB/s]
Archive:  /content/enron-email-dataset.zip
  inflating: emails.csv              


In [4]:
dataset = pd.read_csv('/content/emails.csv', nrows=5000)
dataset_sent_mails = dataset[dataset['file'].str.contains('sent')]
dataset_sent_mails = dataset_sent_mails.assign(sender=dataset_sent_mails["file"].map(lambda x: re.search("(.*)/.*sent", x).group(1)).values)
dataset_sent_mails.drop("file", axis=1, inplace=True)

In [5]:
users = dataset_sent_mails["sender"].value_counts().head(15).index.values # extract top 15 users
mapping = {}
for i, user in enumerate(users, start = 1):
  mapping[user] = i
sent_user_dataset = dataset_sent_mails[dataset_sent_mails.sender.isin(users)] # extracted data of 15 users

In [6]:
sent_user_dataset.head()

,message,sender
0,Message-ID: <18782981.1075855378110.JavaMail.e...,allen-p
1,Message-ID: <15464986.1075855378456.JavaMail.e...,allen-p
2,Message-ID: <24216240.1075855687451.JavaMail.e...,allen-p
3,Message-ID: <13505866.1075863688222.JavaMail.e...,allen-p
4,Message-ID: <30922949.1075863688243.JavaMail.e...,allen-p


In [7]:
sent_user_dataset.shape

(2323, 2)

In [8]:
# preprocessing email content
def email_preprocessing(email_message):
    msg = email.message_from_string(email_message)
    
    email_content = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            email_content.append(part.get_payload())
            
    result = {}
    for key in msg.keys(): 
        result[key] = msg[key]
    result["content"] = ''.join(email_content)
    # msg["content"] = ''.join(email_content)
    return result

#Function for preprocessing of text data
def content_preprocessing(content):
    content = re.sub("[^a-zA-Z]"," ", content)
    words = content.lower().split()
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]

    return ' '.join(words)

In [9]:
final_data = pd.DataFrame(list(map(email_preprocessing, sent_user_dataset.message)))
final_data = pd.DataFrame(list(map(content_preprocessing, final_data[['Subject', 'content']].apply(lambda x: ' '.join(x), axis=1))), columns = ["content"])
final_data = final_data.assign(user_number= sent_user_dataset["sender"].values)
final_data = final_data.replace({'user_number': mapping})
final_data.head()

,content,user_number
0,forecast,1
1,traveling business meeting takes fun trip espe...,1
2,test test successful way go,1
3,randy send schedule salary level everyone sche...,1
4,hello let shoot tuesday,1


In [10]:
final_data.to_csv("preprocessed_Data.csv")

In [11]:
emails_words = final_data.content.apply(lambda x: x.split())

In [12]:
model = Word2Vec(np.array(emails_words))

In [13]:
model.wv.save_word2vec_format('/content/model.bin')

In [14]:
from gensim.models import KeyedVectors
filename = '/content/model.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [15]:
X = final_data.content.values
y_data = final_data.user_number.values
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
#converting into one hot encoded vectors
encoder = OneHotEncoder()
encoder.fit(y_data.reshape(-1, 1))
y_data = encoder.transform(y_data.reshape(-1, 1)).toarray()

In [17]:
X_data = []
max_vec_len = 100
max_seq_len = 100
max_seq_len, max_vec_len

(100, 100)

In [18]:
# Get feature vectors of the word2vec model
for email in emails_words:
  x_arr = []
  for word in email[:max_seq_len]:
    try:
      x_arr.append(model[word])
    except:
      pass
  if max_seq_len - len(x_arr) > 0:
    for _ in range(max_seq_len - len(x_arr)):
      x_arr.append(np.zeros(shape=(max_vec_len,)))
  X_data.append(np.array(x_arr))
  if len(X_data)%5000 == 0:
    print("Next 500 batched finished")
X_data = np.array(X_data)

# np.save('/content/word2vec_data', X_data)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2) #spliting into training and testing

In [20]:
# LSTM model containing encoder LSTM layers and fully connected layers.
class LSTM_Model:
    
    def __init__(self, enc_seq_length, enc_unique_states, output_states, enc_layers=1, 
                 dense__prev_layers_neurons=[], lstm_units = 256, 
                 bidirectional=False, dropout=0, recurrent_dropout=0, bias_regularizer=None, 
                 kernel_regularizer=None, activity_regularizer=None):
        self.enc_seq_length = enc_seq_length
        self.enc_unique_states = enc_unique_states
        self.enc_layers = enc_layers
        self.output_states = output_states
        self.dense__prev_layers_neurons = dense__prev_layers_neurons
        self.lstm_units = lstm_units
        self.bidirectional = bidirectional
        self.dropout = dropout
        self.recurrent_dropout = recurrent_dropout
        self.bias_regularizer = bias_regularizer
        self.kernel_regularizer = kernel_regularizer
        self.activity_regularizer = activity_regularizer
        self.dense__prev_layers_neurons.append(self.output_states)
        
    def getModel(self):
        
        self.encoder_inputs = Input(shape=(None, self.enc_unique_states), name='encoder_inputs')
        
        self.encoder = []
        self.encoder_outputs = []
        
        # Add encoder layers 
        for i in range(self.enc_layers-1):
            self.encoder.append(LSTM(self.lstm_units, 
                                     return_sequences=True, 
                                     recurrent_dropout=self.recurrent_dropout, 
                                     dropout = self.dropout, 
                                     bias_regularizer = self.bias_regularizer, 
                                     activity_regularizer = self.activity_regularizer, 
                                     kernel_regularizer=self.kernel_regularizer, 
                                     name="encoder"+str(i+1)))
            # Wrap Bidirectional layer if bidirectional is True
            if self.bidirectional:
                self.encoder[i] = Bidirectional(self.encoder[i])
            
        self.encoder.append(LSTM(self.lstm_units,  
                                 recurrent_dropout=self.recurrent_dropout, 
                                 dropout = self.dropout, 
                                 bias_regularizer = self.bias_regularizer, 
                                 activity_regularizer = self.activity_regularizer, 
                                 kernel_regularizer=self.kernel_regularizer, 
                                 name="encoder"+str(self.enc_layers)))
        if self.bidirectional:
                self.encoder[self.enc_layers-1] = Bidirectional(self.encoder[self.enc_layers-1])
        
        # Get encoder outputs for each encoder layer
        for i in range(self.enc_layers):
            if i==0:
                self.encoder_outputs.append(self.encoder[i]((self.encoder_inputs)))
            else:
                self.encoder_outputs.append(self.encoder[i](self.encoder_outputs[i-1]))
        
        self.decoder_dense = []
        self.dense_outputs = []
        self.dense_layers = len(self.dense__prev_layers_neurons)
        
        # Add fully connected layers
        for i in range(self.dense_layers):
            if i < self.dense_layers-1:
                self.decoder_dense.append(Dense(self.dense__prev_layers_neurons[i], 
                                                bias_regularizer = self.bias_regularizer, 
                                                activity_regularizer = self.activity_regularizer, 
                                                activation='tanh', name="output_layer"+str(i+1)))
            else:
                self.decoder_dense.append(Dense(self.dense__prev_layers_neurons[i], 
                                                bias_regularizer = self.bias_regularizer, 
                                                activity_regularizer = self.activity_regularizer, 
                                                activation='softmax', name="softmax"))                
            
        # Get outputs of each fully connected layer
        for i in range(self.dense_layers):
            if i==0:
                self.dense_outputs.append(self.decoder_dense[i](self.encoder_outputs[self.enc_layers-1]))
            else:
                self.dense_outputs.append(self.decoder_dense[i](self.dense_outputs[i-1]))
        

        self.model = Model(self.encoder_inputs, self.dense_outputs[self.dense_layers-1])
        
        return self.model

In [21]:
enc_seq_length = max_seq_len
enc_unique_states = max_vec_len
output_states = len(encoder.get_feature_names())
model = LSTM_Model(enc_seq_length, 
              enc_unique_states,
              output_states,
              enc_layers=1,
              lstm_units = 12,
              dense__prev_layers_neurons=[24],
              dropout = 0.1).getModel()

In [22]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_inputs (InputLayer)  [(None, None, 100)]       0         
_________________________________________________________________
encoder1 (LSTM)              (None, 12)                5424      
_________________________________________________________________
output_layer1 (Dense)        (None, 24)                312       
_________________________________________________________________
softmax (Dense)              (None, 2)                 50        
Total params: 5,786
Trainable params: 5,786
Non-trainable params: 0
_________________________________________________________________


In [23]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.002,
    decay_steps=10000,
    decay_rate=0.5)

my_optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

In [24]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
checkpointer = keras.callbacks.ModelCheckpoint("/content/weights_e{epoch:02d}-vl{val_loss:.2f}.hdf5", 
                                       monitor='accuracy',
                                       verbose=1, 
                                       save_best_only=True, 
                                       mode='max')

early_stopping_monitor = keras.callbacks.EarlyStopping(patience=3)

In [26]:
history = model.fit(X_train, y_train, batch_size=64, epochs=5,
                    callbacks=[early_stopping_monitor, checkpointer],
                    validation_split=0.3)

Epoch 1/5
21/21 [==============================] - 22s 45ms/step - loss: 0.6801 - accuracy: 0.6464 - val_loss: 0.6504 - val_accuracy: 0.6505

Epoch 00001: accuracy improved from -inf to 0.66154, saving model to /content/weights_e01-vl0.65.hdf5
Epoch 2/5
21/21 [==============================] - 0s 11ms/step - loss: 0.6337 - accuracy: 0.6794 - val_loss: 0.6485 - val_accuracy: 0.6505

Epoch 00002: accuracy did not improve from 0.66154
Epoch 3/5
21/21 [==============================] - 0s 10ms/step - loss: 0.6413 - accuracy: 0.6593 - val_loss: 0.6473 - val_accuracy: 0.6505

Epoch 00003: accuracy did not improve from 0.66154
Epoch 4/5
21/21 [==============================] - 0s 10ms/step - loss: 0.6400 - accuracy: 0.6622 - val_loss: 0.6481 - val_accuracy: 0.6505

Epoch 00004: accuracy did not improve from 0.66154
Epoch 5/5
21/21 [==============================] - 0s 10ms/step - loss: 0.6488 - accuracy: 0.6497 - val_loss: 0.6472 - val_accuracy: 0.6505

Epoch 00005: accuracy did not improve f

In [30]:
model.save('lstm.h5')

In [31]:
preds = model.predict(X_test)

In [35]:
confusion_matrix(y_test.argmax(axis=1), preds.argmax(axis=1))

array([[286,   0],
       [179,   0]])

In [34]:
accuracy_score(y_test.argmax(axis=1), preds.argmax(axis=1))

0.6150537634408603

In [37]:
!pip freeze > requirements.txt